## Multinomial Naive Bayes

데이타 사이언스 스쿨의 노트북을 참고하여 보다 상세하게 정리한다.[1]

[1]: https://www.datascienceschool.net/view-notebook/c19b48e3c7b048668f2bb0a113bd25f7/#다항-분포-나이브-베이즈-모형

In [1]:
import numpy as np
np.random.seed(0)
X = np.random.randint(2, size=(10, 4))
y = np.array([0,0,0,0,1,1,1,1,1,1])

In [2]:
X

array([[0, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [0, 1, 1, 0]])

In [3]:
y

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [4]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X, y)

In [5]:
# y classes
clf.classes_

array([0, 1])

In [6]:
# y classes의 갯수
clf.class_count_

array([ 4.,  6.])

In [7]:
# 각 y class의 X elements 갯수
fc = clf.feature_count_
fc

array([[ 2.,  4.,  3.,  1.],
       [ 2.,  3.,  5.,  3.]])

In [8]:
# 각 y class의 X elements 합, axis는 값을 합산하는 축
# axis=None(default): elements 전체 합
# axis=0: y 축 기준 합 
fc.sum(axis=0)

array([ 4.,  7.,  8.,  4.])

In [9]:
# axis=1: x 축 기준 합
fc.sum(axis=1)

array([ 10.,  13.])

In [10]:
# newaxis는 길이 1의 새로운 축 생성. None과 동일하다.
fc.sum(axis=1)[:, np.newaxis]

array([[ 10.],
       [ 13.]])

In [11]:
# x 축 기준 합을 반복 축 따라(axis=1) 4회 반복.
np.repeat(fc.sum(axis=1)[:, np.newaxis], 4, axis=1)

array([[ 10.,  10.,  10.,  10.],
       [ 13.,  13.,  13.,  13.]])

In [12]:
# 아래 계산값을 보기 편하도록 한 번 더 출력
fc

array([[ 2.,  4.,  3.,  1.],
       [ 2.,  3.,  5.,  3.]])

In [13]:
# 각 y class의 X elements 갯수를 X elements 합으로 나눈 값이 된다. 
fc / np.repeat(fc.sum(axis=1)[:, np.newaxis], 4, axis=1)

array([[ 0.2       ,  0.4       ,  0.3       ,  0.1       ],
       [ 0.15384615,  0.23076923,  0.38461538,  0.23076923]])

In [14]:
# 라플라스 스무딩 출력
clf.alpha

1.0

In [15]:
# 위 수식에서 라플라스 스무딩을 적용하면 아래와 같다.
(fc + clf.alpha) / (np.repeat(fc.sum(axis=1)[:, np.newaxis], 4, axis=1) + clf.alpha * X.shape[1])

array([[ 0.21428571,  0.35714286,  0.28571429,  0.14285714],
       [ 0.17647059,  0.23529412,  0.35294118,  0.23529412]])

예측은 여러 조건부 확률 $(P(t_k|c),1\leq k\leq n_d)$ 이 곱해지므로 실수형 계산에서 언더플로우 현상이 발생할 수 있다. 이 문제를 개선하기 위해 각 조건부 확률의 로그 값을 구해서 곱하기 대신 더하기를 사용한다. (Manning et al. Introduction to IR. ch13)

딥러닝에서 기울기 소실 문제(Vanishing Gradient Problem)와 유사한 문제인데, 대신 scikit-learn 라이브러리는 `.predict_proba`에서 예측 확률을 표현할때는 전체 로그 확률을 빼주고 지수 함수로 스케일링하여 백분율로 표현한다.

In [16]:
# scikit-learn 라이브러리도 스무딩을 적용한 로그 확률로 계산한다.
clf.feature_log_prob_

array([[-1.54044504, -1.02961942, -1.25276297, -1.94591015],
       [-1.73460106, -1.44691898, -1.04145387, -1.44691898]])

In [17]:
# 로그 확률의 지수 함수를 구하면 상기 확률의 값과 동일함을 확인할 수 있다. 
theta = np.exp(clf.feature_log_prob_)
theta

array([[ 0.21428571,  0.35714286,  0.28571429,  0.14285714],
       [ 0.17647059,  0.23529412,  0.35294118,  0.23529412]])

In [18]:
# 새로운 데이타를 scikit-learn 라이브러리가 예측한 확률
x_new = np.array([1,1,0,0])
clf.predict_proba([x_new])

array([[ 0.55131629,  0.44868371]])

In [19]:
# scikit-learn의 클래스 사전 확률 출력, fit_prior 파라미터이며 디폴트는 True로 적용.
clf.class_log_prior_

array([-0.91629073, -0.51082562])

In [20]:
# 클래스 사전 로그 확률 직접 계산. sckit-learn과 일치한다.
np.log(clf.class_count_) - np.log(clf.class_count_.sum())

array([-0.91629073, -0.51082562])

In [21]:
# 새로운 데이타의 조건부 확률 제곱의 로그 합. 다항 분포(Multinomial Distribution) 적용
# 예를 들어 문장 분류에서 이 값은 문서의 단어 수
np.log(theta ** x_new).sum(axis=1)

array([-2.57006446, -3.18152004])

In [22]:
# 클래스 사전 로그 확률을 더한다.
# IIR 책의 수도 코드는 여기까지의 argmax로 예측을 수행하며, scikit-learn도 동일하다.
# 그러나 예측 확률을 표현하기 위해서는 아래 추가 작업을 진행한다.
log_p = np.log(theta ** x_new).sum(axis=1) + clf.class_log_prior_
log_p

array([-3.48635519, -3.69234566])

In [23]:
# 전체 로그 확률
log_prob_x = np.log(np.exp(log_p).sum())
log_prob_x

-2.8909085872255886

In [24]:
# log_p 값에 전체 로그 확률을 빼고 지수 함수를 구하면 scikit-learn의 예측 확률과 동일한 백분율로 나타냄을 확인할 수 있다.
np.exp(log_p - log_prob_x)

array([ 0.55131629,  0.44868371])